In [63]:
from dotenv import load_dotenv
load_dotenv()
import os

from bs4 import BeautifulSoup
from langchain_community.document_loaders import WebBaseLoader, NewsURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
key = os.getenv("OPENAI_API_KEY")

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent, create_retriever_tool
from langchain_openai.chat_models import ChatOpenAI

In [59]:
from urllib.request import Request, urlopen
import re

# Get the teams transactions page that has signing and extension articles
teamsLoader = WebBaseLoader("https://www.nba.com/news/nba-offseason-every-deal-2024")
teamsSoup = Request('https://www.nba.com/news/nba-offseason-every-deal-2024', headers={'User-Agent':'Mozilla/5.0'})
teamPage = urlopen(teamsSoup).read()

# Get the trades page that has posts on trades that have occured in the offseason
tradesLoader = WebBaseLoader("https://www.nba.com/news/2024-offseason-trade-tracker")
tradesSoup = Request('https://www.nba.com/news/2024-offseason-trade-tracker', headers={'User-Agent':'Mozilla/5.0'})
tradesPage = urlopen(tradesSoup).read()


# Chunking Signings and Extensions

In [254]:
teams = BeautifulSoup(teamPage, 'html.parser')
#print(teams.prettify())

In [255]:
team_posts = teams.find('div',{'class':"Columns_left__XkWXE"})

In [256]:
print(team_posts.prettify())

<div class="Columns_left__XkWXE">
 <div class="ArticleContent_article__NBhQ8">
  <div class="wp-caption alignnone" id="attachment_1385784" style="width: 1930px">
   <img alt="" aria-describedby="caption-attachment-1385784" class="wp-image-1385784 size-full" decoding="async" height="1080" loading="lazy" sizes="(max-width: 1920px) 100vw, 1920px" src="https://cdn.nba.com/manage/2024/06/derrick-white-driving-vs-tyrese-maxey.jpg" srcset="https://cdn.nba.com/manage/2024/06/derrick-white-driving-vs-tyrese-maxey.jpg 1920w, https://cdn.nba.com/manage/2024/06/derrick-white-driving-vs-tyrese-maxey-784x441.jpg 784w, https://cdn.nba.com/manage/2024/06/derrick-white-driving-vs-tyrese-maxey-1568x882.jpg 1568w, https://cdn.nba.com/manage/2024/06/derrick-white-driving-vs-tyrese-maxey-768x432.jpg 768w, https://cdn.nba.com/manage/2024/06/derrick-white-driving-vs-tyrese-maxey-1536x864.jpg 1536w" width="1920"/>
   <p class="wp-caption-text" id="caption-attachment-1385784">
    Derrick White and Tyrese Maxe

In [257]:
headers_to_split_on = [
    ("div", "div 1"),
    ("div", "div 2"),
    ("p", "paragraph"),
    ("strong","s1")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = html_splitter.split_text(str(team_posts))

print(splits[0].page_content)

# for split in splits:
#     print(split.page_content, "\n")

Derrick White and Tyrese Maxey both secured reported deals in the 2024 NBA offseason.  
• Download the NBA App  
The NBA offseason officially began at 12:01 a.m. ET on June 28 following the completion of the 2024 NBA Draft.   
NBA free agency tipped off on June 30 at 6 p.m. ET with players and teams allowed to discuss deals during the moratorium period. Here’s a look at all of the reported and official free-agent deals, extensions and trades for all 30 teams so far.  
Free Agent Tracker | Free Agency Explained | Trade Tracker | 2024 NBA Draft | All NBA Transactions  
Atlanta Hawks  
• Dyson Daniels joins via trade with Pelicans (multiple reports) • Larry Nance Jr. joins via trade with Pelicans (multiple reports)  
> Complete Hawks roster  
Boston Celtics  
• Luke Kornet returns on 1-year deal (per ESPN) • Neemias Queta returns on multi-year deal (per The Athletic) • Jayson Tatum agrees to 5-year max extension (multiple reports) • Derrick White agrees to 4-year extension (multiple repor

In [258]:
team_lines = splits[0].page_content.split("roster")

first_line = team_lines[0].split("All NBA Transactions")
team_lines[0] = first_line[1]

for line in team_lines:
    print(line, "END")

  
Atlanta Hawks  
• Dyson Daniels joins via trade with Pelicans (multiple reports) • Larry Nance Jr. joins via trade with Pelicans (multiple reports)  
> Complete Hawks  END
  
Boston Celtics  
• Luke Kornet returns on 1-year deal (per ESPN) • Neemias Queta returns on multi-year deal (per The Athletic) • Jayson Tatum agrees to 5-year max extension (multiple reports) • Derrick White agrees to 4-year extension (multiple reports)  
> Complete Celtics  END
  
Brooklyn Nets  
• Bojan Bogdanovic joins via trade with Knicks (multiple reports)  
> Complete Nets  END
  
Charlotte Hornets  
• Reggie Jackson joins via trade with Nuggets (multiple reports)  
> Complete Hornets  END
  
Chicago Bulls  
• Josh Giddey joins via trade with Thunder (officially announced) • Jalen Smith agrees to 3-year deal (per ESPN) • Patrick Williams returns on 5-year deal (per The Athletic)  
> Complete Bulls  END
  
Cleveland Cavaliers  
• Donovan Mitchell agrees to 3-year extension (multiple reports)  
> Complete 

# Chunking Trades

In [259]:
trades = BeautifulSoup(tradesPage, 'html.parser')

In [260]:
trade_posts = trades.find('div',{'class':"Columns_left__XkWXE"})
print(trade_posts.prettify())

<div class="Columns_left__XkWXE">
 <div class="ArticleContent_article__NBhQ8">
  <p>
   Keep track of the official trades that happened before, during and after the
   <a href="https://www.nba.com/draft/2024">
    <strong>
     2024 NBA Draft
    </strong>
   </a>
   .
  </p>
  <p>
   <b>
    &gt;
    <a href="https://www.nba.com/news/2024-nba-draft-order">
     Complete 2024 NBA Draft results
    </a>
   </b>
  </p>
  <hr/>
  <p>
   <span style="font-size: 18pt;">
    <a href="https://www.nba.com/news/raptors-kings-jalen-mcdaniels-davion-mitchell-trade-2024-nba-draft">
     <strong>
      Kings land McDaniels from Raptors
     </strong>
    </a>
    (June 28)
   </span>
  </p>
  <p>
   <strong>
    Kings receive:
   </strong>
  </p>
  <ul>
   <li>
    Jalen McDaniels
   </li>
  </ul>
  <p>
   <strong>
    Raptors receive:
   </strong>
  </p>
  <ul>
   <li>
    Davion Mitchell
   </li>
   <li>
    Sasha Vezenkov
   </li>
   <li>
    Jamal Shead (45th pick)
   </li>
  </ul>
  <p>
   <st

In [261]:
headers_to_split_on = [
    ("div", "div 1"),
    ("div", "div 2"),
    ("p", "paragraph"),
    ("strong","s1")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = html_splitter.split_text(str(trade_posts))

print(splits[0].page_content)

# for split in splits:
#     print(split.page_content, "\n")

Keep track of the official trades that happened before, during and after the 2024 NBA Draft.  
> Complete 2024 NBA Draft results  
Kings land McDaniels from Raptors (June 28)  
Kings receive:   
Jalen McDaniels  
Raptors receive:  
Davion Mitchell Sasha Vezenkov Jamal Shead (45th pick)  
Official release: Kings | Raptors  
Pacers, Spurs exchange second-round picks (June 27)  
Pacers receive:  
Johnny Furphy (35th pick)  
Spurs receive:   
Juan Nunez (36th pick) Cash considerations  
Suns add No. 40 pick in deal with Knicks (June 27)  
Suns receive:  
Oso Ighodaro (40th pick)  
Knicks receive:  
Kevin McCullar Jr. (56th pick) 2028 second-round pick  
Official release: Suns  
Thunder deal Waters to Warriors in 3-team deal (June 27)  
Warriors receive:  
Lindy Waters III (via Thunder) Quenten Post (52nd pick, via Thunder from Blazers)  
Thunder receive:  
Draft rights to Quentin Post (via Warriors; traded to Blazers) Draft rights to Oso Ighodaro (40th pick, via Blazers; traded to Suns via

In [262]:
pattern = re.compile(r'(Official release:)[^\|]+\s\|\s[^\|]+')

In [217]:
test = """
Official release: Bulls | Jerks 
So yeah get this thing done pls\xa0
This thing seems weird yea
"""

In [218]:
tests = test.split('\n')

# Filter out lines that match the pattern
filtered_lines = [pattern.sub(r'\1',line) if pattern.match(line) else line for line in tests]

# Join the filtered lines back into a single string
filtered_content = '\n'.join(filtered_lines)

print(filtered_content)


Official release:
So yeah get this thing done pls 
This thing seems weird yea



In [263]:
# trade_lines = splits[0].page_content.split("Official release: ")
trade_lines = str(splits[0].page_content)

trade_lines = trade_lines.strip().split('\n')

# Filter out lines that match the pattern
filtered_lines = [pattern.sub(r'\1',line) if pattern.match(line) else line for line in trade_lines]

trade_lines = filtered_lines[2:]

trade_lines = ' \n'.join(trade_lines)

trade_lines = trade_lines.split("Official release: ")

for line in trade_lines:
    print(line, "END")

Kings land McDaniels from Raptors (June 28)   
Kings receive:    
Jalen McDaniels   
Raptors receive:   
Davion Mitchell Sasha Vezenkov Jamal Shead (45th pick)   
 END

Pacers, Spurs exchange second-round picks (June 27)   
Pacers receive:   
Johnny Furphy (35th pick)   
Spurs receive:    
Juan Nunez (36th pick) Cash considerations   
Suns add No. 40 pick in deal with Knicks (June 27)   
Suns receive:   
Oso Ighodaro (40th pick)   
Knicks receive:   
Kevin McCullar Jr. (56th pick) 2028 second-round pick   
 END
Suns   
Thunder deal Waters to Warriors in 3-team deal (June 27)   
Warriors receive:   
Lindy Waters III (via Thunder) Quenten Post (52nd pick, via Thunder from Blazers)   
Thunder receive:   
Draft rights to Quentin Post (via Warriors; traded to Blazers) Draft rights to Oso Ighodaro (40th pick, via Blazers; traded to Suns via Knicks)   
Blazers receive:   
Draft rights to Quentin Post (via Thunder; traded to Warriors) Cash considerations (via Thunder)   
 END

Pelicans acquire

# Embedding for ChatGPT

In [264]:
# OpenAI Embeddings
embeddings = OpenAIEmbeddings()

In [303]:
# Convert lists into langchain docs
from langchain.docstore.document import Document

trade_documents = [
    Document(page_content=content)
    for content in trade_lines
]

team_documents = [
    Document(page_content=content)
    for content in team_lines
]

documents = trade_documents + team_documents
documents = documents[12:42]

In [305]:
print(documents[29].page_content)

  
Washington Wizards  
• Malcolm Brogdon joins via trade with Trail Blazers (multiple reports) • Richaun Holmes returns on 2-year deal (per The Athletic) • Jonas Valanciunas agrees to 3-year deal (per ESPN)  
> Complete Wizards 


In [306]:
db = FAISS.from_documents(documents, embeddings)

In [307]:
retreiver = db.as_retriever()
tool = create_retriever_tool(
    retreiver,
    "search_documents",
    "Search and return information from the documents relating to the trades, signings, and extensions"
)
tools = [tool]

In [308]:
llm = ChatOpenAI(temperature=0)
agent_executor = create_conversational_retrieval_agent(
    llm,
    tools,
    verbose=True
)

In [ ]:
def ask(q):
    template = """"""

In [309]:
input = "What free agents have the Bulls signed this offseason?"
result = agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...

Invoking: `search_documents` with `{'query': 'Bulls free agent signings offseason'}`


  
Chicago Bulls  
• Josh Giddey joins via trade with Thunder (officially announced) • Jalen Smith agrees to 3-year deal (per ESPN) • Patrick Williams returns on 5-year deal (per The Athletic)  
> Complete Bulls 

  
Los Angeles Clippers  
• Mo Bamba agrees to 1-year deal (per ESPN) • Nicolas Batum agrees to a 2-year deal (per ESPN) • Kris Dunn agrees to deal (multiple reports) • James Harden returns on 2-year deal (multiple reports) • Derrick Jones Jr. agrees to 3-year deal (per The Athletic) • Kevin Porter Jr. agrees to 2-year deal (per The Athletic)  
> Complete Clippers 

  
Golden State Warriors  
• De’Anthony Melton agrees to 1-year deal (per The Athletic) • Chris Paul to be waived, become free agent (officially announced) • Lindy Waters III joins via trade with Thunder (officially announced)  
> Complete Warriors 

  
Phoenix Suns  
• Bol Bol returns on

TypeError: expected string or buffer

In [238]:
print(result)

NameError: name 'result' is not defined